In [1]:
import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")

import nltk
from sklearn.feature_extraction.text import TfidfVectorizer

import warnings
warnings.filterwarnings("ignore")

In [20]:
data_train_full = pd.read_csv('./transformed_data/data_train_after_preprocessing.csv', sep=';')
data_test_full = pd.read_csv('./transformed_data/data_test_after_preprocessing.csv', sep=';')
data_val_full = pd.read_csv('./transformed_data/data_val_after_preprocessing.csv', sep=';')

In [21]:
unuse = ['Unnamed: 0', 'Unnamed: 0.1']

data_train_full = data_train_full.drop(columns = unuse)
data_test_full = data_test_full.drop(columns = unuse)
data_val_full = data_val_full.drop(columns = unuse)

In [22]:
data_train_full.head()

,text,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,...,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral,text_transformed
0,Don't be coy were talking about the active ing...,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"['', '', '', 'coy', '', 'talking', '', '', 'ac..."
1,"It's likely that they don't have much, yet sti...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"['', '', 'likely', '', '', '', '', '', 'much',..."
2,Sure but the news article with [NAME] vanishin...,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"['sure', '', '', 'news', 'article', '', 'name'..."
3,i would not vote for him,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"['', 'would', '', 'vote', '', '']"
4,Funny way to oppose fascism - since they are t...,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"['funny', 'way', '', 'oppose', 'fascism', 'sin..."


In [26]:
data_train_full['Index'] = range(1, len(data_train_full) + 1)
data_test_full['Index'] = range(1, len(data_test_full) + 1)
data_val_full['Index'] = range(1, len(data_val_full) + 1)
data_train_full = data_test_full.set_index(['Index'])
data_test_full = data_test_full.set_index(['Index'])
data_val_full = data_val_full.set_index(['Index'])

In [27]:
data_test_full.head()

,text,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,...,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral,text_transformed
Index,,,,,,,,,,,,,,,,,,,,,
1,Literally everywhere. There's nothing especial...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"['literally', 'everywhere', '', '', 'nothing',..."
2,"[NAME] and [NAME] may be stronger, but [NAME] ...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,"['name', '', 'name', 'may', '', 'stronger', ''..."
3,Ik I’m crying rn,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,"['ik', '', '', 'crying', 'rn']"
4,"He was cut yesterday, unfortunately",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,"['', '', 'cut', 'yesterday', 'unfortunately']"
5,"Nice, I saw them during the Demo and was prett...",0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"['nice', '', 'saw', '', '', '', 'demo', '', ''..."


In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [25]:
X_train_full = data_train_full['text_transformed']
X_test_full = data_test_full['text_transformed']
X_val_full = data_val_full['text_transformed']
y_train_full = data_train_full.drop(['text','text_transformed'],axis=1)
y_test_full = data_test_full.drop(['text','text_transformed'],axis=1)
y_val_full = data_val_full.drop(['text','text_transformed'],axis=1)

In [17]:
def tfidf(X_train_,X_test_,y_train_,y_test_,X_val_,y_val_,target,max_features = 2000):
    ngram_range = (1,1)
    min_df = 1
    max_df = 1.
    tfidf = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        max_features=max_features,
                        norm='l2')
    features_train = tfidf.fit_transform(X_train_).toarray()
    labels_train = y_train_.values
    features_test = tfidf.transform(X_test_).toarray()
    labels_test = y_test_.values
    features_val = tfidf.transform(X_val_).toarray()
    labels_val = y_val_.values
    features_all = tfidf.get_feature_names()
    data_train_tfidf = pd.DataFrame(data=features_train,columns=features_all)
    data_train_tfidf = data_train_tfidf.join(pd.DataFrame(data=labels_train,columns=target))
    data_test_tfidf = pd.DataFrame(data=features_test,columns=features_all)
    data_test_tfidf = data_test_tfidf.join(pd.DataFrame(data=labels_test,columns=target))
    data_val_tfidf = pd.DataFrame(data=features_val,columns=features_all)
    data_val_tfidf = data_val_tfidf.join(pd.DataFrame(data=labels_val,columns=target))
    return data_train_tfidf,data_test_tfidf,data_val_tfidf

In [18]:
target = data_train_full.drop(['text','text_transformed'],axis = 1).columns

In [19]:
data_train_full_tfidf,data_test_full_tfidf,data_val_full_tfidf = tfidf(X_train_full,X_test_full,y_train_full,y_test_full,X_val_full,y_val_full,target,2000)

ValueError: columns overlap but no suffix specified: Index(['caring', 'fear', 'joy', 'love', 'surprise'], dtype='object')